## Topic Detect Model

In [3]:
sentence_bert_path = '../src/models/text2vec-base-chinese'

In [7]:
from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn.functional as F
import os

class SentenceBert():
    def __init__(self, embed_model):
        super().__init__()
        self.embedding = AutoModel.from_pretrained(embed_model)
        self.tokenizer = AutoTokenizer.from_pretrained(embed_model)

    def forward(self, sentences):
        inputs = self.tokenizer(
            sentences,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=512
        )

        with torch.no_grad():
            outputs = self.embedding(**inputs)

        token_embed = outputs.last_hidden_state      # (Batch, T, H)
        attention_mask = inputs['attention_mask']    # (Batch, T)

        mask = attention_mask.unsqueeze(-1).expand(token_embed.size()).float()
        sentence_embed = (token_embed * mask).sum(dim=1) / mask.sum(dim=1)
        sentence_embed = F.normalize(sentence_embed, p=2, dim=1)

        return sentence_embed

model = SentenceBert(embed_model=sentence_bert_path)

In [8]:
import torch
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np

class SentenceEmbeddingModel():
    def __init__(self, model_path, device=None):
        super().__init__()
        self.model_path = model_path
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"使用设备: {self.device}")
        
        # 使用sentence_transformers，支持GPU加速
        self.model = SentenceTransformer(model_path, device=self.device)
    
    def forward(self, sentences, batch_size=64, show_progress=True):
        """
        批量生成embeddings，支持进度显示
        
        Args:
            sentences: 文本列表
            batch_size: 批处理大小，GPU可用时可以设为64-128
            show_progress: 是否显示进度条
        """
        embeddings = self.model.encode(
            sentences,
            batch_size=batch_size,
            show_progress_bar=show_progress,
            normalize_embeddings=True,
            device=self.device
        )
        return embeddings

In [9]:
# Demo to test ShortTopicDetect class

sentences = ['今天的天气真好', '今天天气不错']
outputs = model.forward(sentences)
print('Embedding outputs:\n', outputs)
print('\n', outputs.shape)
print(outputs[0] @ outputs[1])  # Cosine similarity between two sentence embeddings

Embedding outputs:
 tensor([[-0.0115, -0.0204,  0.0322,  ...,  0.0142,  0.0103, -0.0010],
        [ 0.0088, -0.0345,  0.0342,  ...,  0.0062,  0.0138,  0.0177]])

 torch.Size([2, 768])
tensor(0.8632)


### HDBSCAN Model

In [10]:
import hdbscan
import numpy as np
from collections import defaultdict
from sklearn.preprocessing import normalize


class HDBSCAN():
    def __init__(self, min_cluster_size=100, min_samples=None):
        """
        Args:
            min_cluster_size: 最小聚类大小，建议50-200，数据量大时调大
            min_samples: 最小样本数，默认为min_cluster_size的一半
        """
        self.min_cluster_size = min_cluster_size
        self.min_samples = min_samples if min_samples else min_cluster_size // 2

        self.clusterer = hdbscan.HDBSCAN(
            min_cluster_size=self.min_cluster_size,
            min_samples=self.min_samples,
            metric='euclidean',  # 归一化后的向量，euclidean等价于cosine
            prediction_data=True,
            cluster_selection_method='eom'
        )
        self.labels_ = None
        self.topic_centers_ = {}
        self.topic_sizes_ = {}
    
    def fit(self, embeddings):
        # 归一化embeddings，使其在单位球面上
        embeddings = normalize(embeddings)

        self.clusterer.fit(embeddings)
        self.labels_ = self.clusterer.labels_
        self._compute_topic_centers(embeddings)

        return self.labels_

    def _compute_topic_centers(self, embeddings):
        topic_vectors = defaultdict(list)

        for vec, label in zip(embeddings, self.labels_):
            if label == -1:
                continue  # 忽略噪声
            topic_vectors[label].append(vec)

        self.topic_centers_ = {}
        self.topic_sizes_ = {}

        for label, vecs in topic_vectors.items():
            vecs = np.vstack(vecs)
            center = vecs.mean(axis=0)
            center = center / np.linalg.norm(center)

            self.topic_centers_[label] = center
            self.topic_sizes_[label] = len(vecs)
    
    def get_topics(self):
        topics = []
        for label in self.topic_centers_:
            topics.append({
                "topic_id": label,
                "size": self.topic_sizes_[label],
                "center": self.topic_centers_[label]
            })
        # 按size排序
        topics.sort(key=lambda x: x['size'], reverse=True)
        return topics

    def predict(self, new_embeddings):
        new_embeddings = normalize(new_embeddings)

        labels, _ = hdbscan.approximate_predict(
            self.clusterer,
            new_embeddings
        )

        return labels

### Experiment on Weibo data

In [11]:
import json
import pandas as pd

data_path = '../html/data/weibo_data.json'
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"总数据量: {len(data)} 条")
print("\n示例数据:")
print(json.dumps(data[0], ensure_ascii=False, indent=2))

总数据量: 30341 条

示例数据:
{
  "source": "weibo",
  "topic": "%E7%BA%A2%E5%8C%85",
  "weibo_id": "5266154761489193",
  "text": "#情人节微信开放520元红包# 都去试一下，今天真的可以转520的大红包诶 ",
  "url": "https://weibo.com/1810074673/5266154761489193",
  "timestamp": "Sat Feb 14 09:29:58 +0800 2026",
  "reposts": 8,
  "comments": 49,
  "likes": 177
}


In [12]:
import numpy as np
from collections import Counter

# 提取文本内容
texts = [item['text'] for item in data if item.get('text')]
print(f"有效文本数: {len(texts)} 条")

# 先看看数据的来源分布
topic_distribution = {}
for item in data:
    topic = item.get('topic', 'unknown')
    topic_distribution[topic] = topic_distribution.get(topic, 0) + 1

print(f"\n数据来源 (按搜索关键词):")
for topic, count in sorted(topic_distribution.items(), key=lambda x: x[1], reverse=True)[:10]:
    from urllib.parse import unquote
    print(f"  {unquote(topic)}: {count} 条")

# 初始化模型并生成embeddings
model = SentenceEmbeddingModel(model_path=sentence_bert_path)

# GPU可用时用更大的batch_size加速
batch_size = 256
embeddings = model.forward(texts, batch_size=batch_size, show_progress=True)

print(f"\nEmbeddings shape: {embeddings.shape}")

# ===== 方案1: 对所有数据聚类 =====
print("\n========== 方案1: 全局聚类 ==========")
# 设置更合理的 min_cluster_size
for min_cluster_size in [100, 200, 300]:
    detector = HDBSCAN(min_cluster_size=min_cluster_size)
    labels = detector.fit(embeddings)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)
    print(f"min_cluster_size={min_cluster_size}: {n_clusters} clusters, {n_noise} noise ({n_noise/len(labels)*100:.1f}%)")

# 用 min_cluster_size=200 做最终分析
detector = HDBSCAN(min_cluster_size=200)
labels = detector.fit(embeddings)
topics = detector.get_topics()

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = list(labels).count(-1)

print(f"\n========== 聚类结果 ==========")
print(f"总数据量: {len(labels)}")
print(f"发现topic数: {n_clusters}")
print(f"噪声数据: {n_noise} 条 ({n_noise/len(labels)*100:.1f}%)")

# 显示top topics
if n_clusters > 0:
    label_counts = Counter(labels)
    print(f"\n========== Top 10 Topics ==========")
    sorted_topics = sorted(label_counts.items(), key=lambda x: x[1], reverse=True)
    for i, (label, count) in enumerate(sorted_topics[:11]):
        if label == -1:
            continue
        print(f"Topic {label}: {count} 条")

    # 为每个top topic找代表性文本
    print(f"\n========== Top 5 Topics 代表性文本 ==========")
    for i, topic_info in enumerate(topics[:5]):
        topic_id = topic_info['topic_id']
        center = topic_info['center']
        
        # 找到属于该topic的所有文本索引
        topic_indices = [i for i, label in enumerate(labels) if label == topic_id]
        
        if topic_indices:
            # 计算与中心的相似度
            topic_embeddings = embeddings[topic_indices]
            similarities = topic_embeddings @ center
            
            # 找到最相似的3条文本
            top_3_idx = np.argsort(similarities)[-3:][::-1]
            print(f"\n--- Topic {topic_id} (共{len(topic_indices)}条) ---")
            for idx in top_3_idx:
                original_idx = topic_indices[idx]
                print(f"  [{idx+1}] {texts[original_idx][:80]}...")

# ===== 方案2: 按热搜关键词分组分析 =====
print(f"\n\n========== 方案2: 按热搜关键词分组细分 ==========")
from urllib.parse import unquote

# 选择数据量最多的3个关键词进行细分分析
top_keywords = sorted(topic_distribution.items(), key=lambda x: x[1], reverse=True)[:3]

for keyword, count in top_keywords:
    print(f"\n--- 热搜: {unquote(keyword)} (共{count}条) ---")
    
    # 提取该关键词下的所有文本
    keyword_indices = [i for i, item in enumerate(data) if item.get('topic') == keyword]
    keyword_texts = [texts[i] for i in keyword_indices]
    keyword_embeddings = embeddings[keyword_indices]
    
    # 对该关键词下的内容进行聚类
    # min_cluster_size 设为该关键词数据量的 10-20%
    mcs = max(5, int(len(keyword_texts) * 0.1))
    detector_kw = HDBSCAN(min_cluster_size=mcs)
    labels_kw = detector_kw.fit(keyword_embeddings)
    
    n_clusters_kw = len(set(labels_kw)) - (1 if -1 in labels_kw else 0)
    n_noise_kw = list(labels_kw).count(-1)
    
    print(f"  细分出 {n_clusters_kw} 个子话题, 噪声 {n_noise_kw} 条")
    
    # 显示该关键词下的子话题
    if n_clusters_kw > 0:
        topics_kw = detector_kw.get_topics()
        for j, topic_info in enumerate(topics_kw[:3]):
            topic_id = topic_info['topic_id']
            center = topic_info['center']
            topic_indices_kw = [k for k, label in enumerate(labels_kw) if label == topic_id]
            
            # 找代表性文本
            topic_embs = keyword_embeddings[topic_indices_kw]
            sims = topic_embs @ center
            best_idx = topic_indices_kw[sims.argmax()]
            
            print(f"    子话题{j+1} ({len(topic_indices_kw)}条): {keyword_texts[best_idx][:60]}...")

有效文本数: 30341 条

数据来源 (按搜索关键词):
  微信回应红包手气最佳攻略: 1250 条
  看电影的快乐正全面升级: 1225 条
  红包: 1224 条
  情人节: 1200 条
  2026NBA全明星: 1125 条
  生命树豆瓣开分8.1: 1100 条
  檀健次卢昱晓剧宣: 1081 条
  突然发现朋友圈没人晒花了: 1058 条
  黄子韬借综艺扇了徐艺洋: 1000 条
  暖阳发文: 975 条
使用设备: cuda


Batches: 100%|██████████| 119/119 [11:33<00:00,  5.83s/it]



Embeddings shape: (30341, 768)

========== 方案1: 全局聚类 ==========
min_cluster_size=100: 60 clusters, 6413 noise (21.1%)
min_cluster_size=200: 34 clusters, 8888 noise (29.3%)
min_cluster_size=300: 25 clusters, 10986 noise (36.2%)

========== 聚类结果 ==========
总数据量: 30341
发现topic数: 34
噪声数据: 8888 条 (29.3%)

========== Top 10 Topics ==========
Topic 31: 1490 条
Topic 9: 1100 条
Topic 17: 1035 条
Topic 23: 975 条
Topic 25: 951 条
Topic 7: 924 条
Topic 1: 900 条
Topic 0: 860 条
Topic 13: 851 条
Topic 24: 840 条

========== Top 5 Topics 代表性文本 ==========

--- Topic 31 (共1490条) ---
  [855] #网友偶遇了易烊千玺#好帅啊啊啊啊大年初一看《惊蛰无声》！！！ ...
  [199] #网友偶遇了易烊千玺#好帅啊啊啊啊大年初一看《惊蛰无声》！！！ ...
  [1429] #网友偶遇了易烊千玺#好帅啊啊啊啊大年初一看《惊蛰无声》！！！ ...

--- Topic 9 (共1100条) ---
  [1071] #生命树豆瓣开分8.1#这么好的剧值得9分以上，那行粉圈的黑子爬远一点！生命树杨紫 ...
  [46] #生命树豆瓣开分8.1#这么好的剧值得9分以上，那行粉圈的黑子爬远一点！生命树杨紫 ...
  [96] #生命树豆瓣开分8.1#这么好的剧值得9分以上，那行粉圈的黑子爬远一点！生命树杨紫 ...

--- Topic 17 (共1035条) ---
  [8] NBA王鹤棣-我的评分：#2026NBA全明星##NBA全明星名人赛# ...
  [113] NBA王鹤棣-我的评分：#2026NBA全明星##NBA全明星名